In [1]:
import pandas as pd
import datetime
import time

# Import des données

In [2]:
ticket_data = pd.read_csv('ticket_data.csv')
cities = pd.read_csv('cities.csv')
stations = pd.read_csv('stations.csv')
providers = pd.read_csv('providers.csv')

In [3]:
stations.head() # voir les 5 premières lignes

,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


In [4]:
cities.head() # voir les 5 premières lignes

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [5]:
providers.head()  # voir les 5 premières lignes

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [6]:
# ticket_data.loc[:,'id']
print(ticket_data.dtypes)
ticket_data.head() # voir les 5 premières lignes

id                   int64
company              int64
o_station          float64
d_station          float64
departure_ts        object
arrival_ts          object
price_in_cents       int64
search_ts           object
middle_stations     object
other_companies     object
o_city               int64
d_city               int64
dtype: object


,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


#### Ajouter la colonne duration pour chaque ticket

In [7]:
ticket_data2 = ticket_data.copy()  

In [8]:
# Reconversion de la chaîne en date-temps (departure_ts, arrival_ts )
ticket_data2['departure_ts'] = pd.to_datetime(ticket_data2['departure_ts'], format='%Y-%m-%d %H:%M:%S+%f', errors='coerce')
ticket_data2['arrival_ts'] = pd.to_datetime(ticket_data2['arrival_ts'], format='%Y-%m-%d %H:%M:%S+%f', errors='coerce')

# Caculer la duration pour chaque ticket
ticket_data2['duration'] = ticket_data2['arrival_ts'] - ticket_data2['departure_ts'] 

#Test
print(ticket_data2.dtypes)
ticket_data2.head()

id                           int64
company                      int64
o_station                  float64
d_station                  float64
departure_ts        datetime64[ns]
arrival_ts          datetime64[ns]
price_in_cents               int64
search_ts                   object
middle_stations             object
other_companies             object
o_city                       int64
d_city                       int64
duration           timedelta64[ns]
dtype: object


,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,duration
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00,2017-10-13 20:10:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542,0 days 06:10:00
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542,0 days 17:50:00
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542,1 days 07:57:00
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542,0 days 21:35:00
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542,0 days 21:46:00


## 1. Extraire des infos intéressantes type prix min, moyen et max, durée min/max/moyenne par trajet

#### En observant les données, on détermine un trajet par ces gare d'origine (o_station) et station de destination (d_station)

### Le prix min, moyen et max par trajet

In [9]:
# Caculer le prix min, moyen et max par trajet
prix_trajet_station = ticket_data2.groupby(['o_station','d_station'])["price_in_cents"].agg(['min', 'mean', 'max'])

## Changer les noms des ces nouvelles colonnes
prix_trajet_station.rename(columns={'min':'min_price_in_cents', 'mean':'mean_price_in_cents', 'max':'max_price_in_cents'}, inplace=True)

# Round 
decimals = pd.Series([0], index=['mean_price_in_cents'])
prix_trajet_station = prix_trajet_station.round(decimals)

#Test
prix_trajet_station.head()
# type(prix_trajet_station)

min_price_in_cents  mean_price_in_cents  \
o_station d_station                                            
3.0       39.0                     1200               2238.0   
          61.0                     4780               4780.0   
          123.0                    3600               3600.0   
          132.0                    1890               2150.0   
          189.0                    3680               3680.0   

                     max_price_in_cents  
o_station d_station                      
3.0       39.0                     3480  
          61.0                     4780  
          123.0                    3600  
          132.0                    2280  
          189.0                    3680

### Le durée min/max/moyenne par trajet

In [10]:
print(ticket_data2.dtypes)

id                           int64
company                      int64
o_station                  float64
d_station                  float64
departure_ts        datetime64[ns]
arrival_ts          datetime64[ns]
price_in_cents               int64
search_ts                   object
middle_stations             object
other_companies             object
o_city                       int64
d_city                       int64
duration           timedelta64[ns]
dtype: object


In [11]:
# Caculer le durée min, moyen et max par trajet
ticket_data2['duration'] = ticket_data2['duration'].dt.total_seconds()
duration_trajet_station = ticket_data2.groupby(['o_station','d_station'])['duration'].agg(['min', 'mean', 'max'])

## Changer les noms des ces nouvelles colonnes
duration_trajet_station.rename(columns={'min':'min_duration_in_seconds', 'mean':'mean_duration_in_seconds', 'max':'max_duration_in_seconds'}, inplace=True)

duration_trajet_station.head()

min_duration_in_seconds  mean_duration_in_seconds  \
o_station d_station                                                      
3.0       39.0                       22500.0                   25650.0   
          61.0                       69300.0                   69300.0   
          123.0                      81000.0                   81000.0   
          132.0                      18000.0                   23400.0   
          189.0                      52500.0                   52500.0   

                     max_duration_in_seconds  
o_station d_station                           
3.0       39.0                       34500.0  
          61.0                       69300.0  
          123.0                      81000.0  
          132.0                      33300.0  
          189.0                      52500.0

#### Jointure df prix_trajet et duration_trajet

In [12]:
trajet_station = prix_trajet_station.join(duration_trajet_station, on=['o_station','d_station'])
#Test
trajet_station.head()

min_price_in_cents  mean_price_in_cents  \
o_station d_station                                            
3.0       39.0                     1200               2238.0   
          61.0                     4780               4780.0   
          123.0                    3600               3600.0   
          132.0                    1890               2150.0   
          189.0                    3680               3680.0   

                     max_price_in_cents  min_duration_in_seconds  \
o_station d_station                                                
3.0       39.0                     3480                  22500.0   
          61.0                     4780                  69300.0   
          123.0                    3600                  81000.0   
          132.0                    2280                  18000.0   
          189.0                    3680                  52500.0   

                     mean_duration_in_seconds  max_duration_in_seconds  
o_station d_station                                                     
3.0       39.0                        25650.0                  34500.0  
          61.0                        69300.0                  69300.0  
          123.0                       81000.0                  81000.0  
          132.0                       23400.0                  33300.0  
          189.0                       52500.0                  52500.0

#### Ajouter les autres infomations concernants aux stations en utilisant fct Merge

##### Ajouter les noms des stations pour chaque trajet

In [13]:
# Préparer des données pour le fct Join d'après
stations_name = stations.loc[:,['id','unique_name']]
stations_name = stations_name.set_index('id')

# Ajouter la nom de la gare d'origine
trajet_station = trajet_station.join(stations_name, on='o_station')
## Changer le nom de la colonne 'unique_name' par 'o_station_name'
trajet_station.rename(columns={'unique_name':'o_station_name'}, inplace=True)

# Ajouter la nom de station de destination
trajet_station = trajet_station.join(stations_name, on='d_station')
## Changer le nom de la colonne 'unique_name' par 'd_station_name'
trajet_station.rename(columns={'unique_name':'d_station_name'}, inplace=True)

#Test
trajet_station.head()

min_price_in_cents  mean_price_in_cents  \
o_station d_station                                            
3.0       39.0                     1200               2238.0   
          61.0                     4780               4780.0   
          123.0                    3600               3600.0   
          132.0                    1890               2150.0   
          189.0                    3680               3680.0   

                     max_price_in_cents  min_duration_in_seconds  \
o_station d_station                                                
3.0       39.0                     3480                  22500.0   
          61.0                     4780                  69300.0   
          123.0                    3600                  81000.0   
          132.0                    2280                  18000.0   
          189.0                    3680                  52500.0   

                     mean_duration_in_seconds  max_duration_in_seconds  \
o_station d_station                                                      
3.0       39.0                        25650.0                  34500.0   
          61.0                        69300.0                  69300.0   
          123.0                       81000.0                  81000.0   
          132.0                       23400.0                  33300.0   
          189.0                       52500.0                  52500.0   

                    o_station_name                             d_station_name  
o_station d_station                                                            
3.0       39.0       Aéroport CDG              Antwerp (Koningin Astridplein)  
          61.0       Aéroport CDG                            Avenue du Campon  
          123.0      Aéroport CDG                                 Bremerhaven  
          132.0      Aéroport CDG                     Bruges (Station Brugge)  
          189.0      Aéroport CDG   Dole (Parking next to the tollgate/péage)

##### Ajouter les ids des villes pour chaque trajet

In [14]:
city_trajet = ticket_data[['o_station', 'd_station', 'o_city','d_city']]
city_trajet  = city_trajet.drop_duplicates()
city_trajet  = city_trajet.dropna()
print(city_trajet.head())

trajet_station = city_trajet.join(trajet_station, on=['o_station', 'd_station'])
trajet_station.head()

    o_station  d_station  o_city  d_city
1        63.0     1044.0     611     542
2      5905.0     6495.0     611     542
49      767.0      396.0     628     453
50    10141.0      396.0     628     453
51    10144.0      396.0     628     453


,o_station,d_station,o_city,d_city,min_price_in_cents,mean_price_in_cents,max_price_in_cents,min_duration_in_seconds,mean_duration_in_seconds,max_duration_in_seconds,o_station_name,d_station_name
1,63.0,1044.0,611,542,1450,1450.0,1450,64200.0,64200.000000,64200.0,Avenue Georges Pompidou,Station Les Sabines
2,5905.0,6495.0,611,542,7400,9537.0,13500,77700.0,90360.000000,115020.0,Les Aubrais,Montpellier
49,767.0,396.0,628,453,1800,2002.0,2390,40500.0,57987.804878,134400.0,Massy-Palaiseau,Gare Lille-Europe
50,10141.0,396.0,628,453,1800,2009.0,2390,22500.0,32962.025316,121200.0,Paris Ibis Clichy Batignolles,Gare Lille-Europe
51,10144.0,396.0,628,453,1100,1828.0,2390,22800.0,62374.647887,111000.0,Paris Nord - St-Denis université,Gare Lille-Europe


##### Ajouter les noms des villes pour chaque trajet

In [15]:
# Préparer des données pour le fct Join d'après
cities_name = cities.loc[:,['id','unique_name']]
cities_name = cities_name.set_index('id')
#cities_name.head()

# Ajouter la nom de la vile d'origine
trajet_station = trajet_station.join(cities_name, on ='o_city')
trajet_station.head()
## Changer le nom de cette nouvelle colonne
trajet_station.rename(columns={'unique_name':'o_city_name'}, inplace=True)

# Ajouter la nom de la ville de destination
trajet_station = trajet_station.join(cities_name, on='d_city')
## Changer le nom de cette nouvelle colonne
trajet_station.rename(columns={'unique_name':'d_city_name'}, inplace=True)

#Test
trajet_station.head()

,o_station,d_station,o_city,d_city,min_price_in_cents,mean_price_in_cents,max_price_in_cents,min_duration_in_seconds,mean_duration_in_seconds,max_duration_in_seconds,o_station_name,d_station_name,o_city_name,d_city_name
1,63.0,1044.0,611,542,1450,1450.0,1450,64200.0,64200.000000,64200.0,Avenue Georges Pompidou,Station Les Sabines,orleans,montpellier
2,5905.0,6495.0,611,542,7400,9537.0,13500,77700.0,90360.000000,115020.0,Les Aubrais,Montpellier,orleans,montpellier
49,767.0,396.0,628,453,1800,2002.0,2390,40500.0,57987.804878,134400.0,Massy-Palaiseau,Gare Lille-Europe,paris,lille
50,10141.0,396.0,628,453,1800,2009.0,2390,22500.0,32962.025316,121200.0,Paris Ibis Clichy Batignolles,Gare Lille-Europe,paris,lille
51,10144.0,396.0,628,453,1100,1828.0,2390,22800.0,62374.647887,111000.0,Paris Nord - St-Denis université,Gare Lille-Europe,paris,lille


## 2. Différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km)


In [16]:
ticket_data2.head() # voir les 5 premières lignes

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,duration
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00,2017-10-13 20:10:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542,22200.0
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542,64200.0
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542,115020.0
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542,77700.0
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542,78360.0


In [17]:
providers.head(10) # voir les 10 premières lignes

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus
5,14,6,NaN,driivme,DriiveMe,False,False,False,False,car
6,15,8,NaN,alsa,Alsa,False,False,False,False,bus
7,16,9,NaN,distribusion,Distribusion,False,False,False,False,bus
8,17,9,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus
9,18,9,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus


##### Ajouter les providers pour chaque ticket

In [18]:
# Préparer des données pour le fct Join d'après
#providers_info = providers.loc[:,['id', 'fullname', 'transport_type']]
providers_info = providers.set_index('id')

# Ajouter la nom du fournisseur et le moyenne de transport
ticket_provider = ticket_data2.join(providers_info, on ='company')
## Changer le nom de cette nouvelle colonne
ticket_provider.rename(columns={'fullname':'provider_name'}, inplace=True)

#Test
ticket_provider.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,duration,company_id,provider_id,name,provider_name,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00,2017-10-13 20:10:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,...,22200.0,11,NaN,bbc,Blablacar,False,False,False,False,carpooling
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,64200.0,1,NaN,ouibus,Ouibus,True,True,True,False,bus
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",...,115020.0,10,2,corailintercite,Intercités,False,False,True,False,train
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",...,77700.0,10,2,corailintercite,Intercités,False,False,True,False,train
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},...,78360.0,10,6,coraillunea,Intercités de Nuit,False,False,True,False,train


##### Ajouter les positions pour chaque ticket selon les coordonnées des ces stations

In [19]:
stations2 = stations.copy() 
stations2 = stations2.set_index('id')
#stations2.head()

# Ajouter la nom de la vile d'origine
ticket_provider = ticket_provider.join(stations2, on ='o_station')
## Changer le nom de cette nouvelle colonne
ticket_provider.rename(columns={'unique_name':'o_station_name', 'latitude':'o_station_lat','longitude':'o_station_lon'}, inplace=True)

# Ajouter la nom de la ville de destination
ticket_provider = ticket_provider.join(stations2, on='d_station')
## Changer le nom de cette nouvelle colonne
ticket_provider.rename(columns={'unique_name':'d_station_name','latitude':'d_station_lat','longitude':'d_station_lon'}, inplace=True)

#Test
ticket_provider.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,has_plug,has_adjustable_seats,has_bicycle,transport_type,o_station_name,o_station_lat,o_station_lon,d_station_name,d_station_lat,d_station_lon
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00,2017-10-13 20:10:00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,...,False,False,False,carpooling,NaN,NaN,NaN,NaN,NaN,NaN
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,True,True,False,bus,Avenue Georges Pompidou,47.896911,1.854277,Station Les Sabines,43.583932,3.860233
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500


##### Supprimer les lignes invalides avant de faire les calculs

In [20]:
ticket_provider = ticket_provider.dropna(subset=['o_station','d_station','transport_type'])       

# Test
ticket_provider.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,has_plug,has_adjustable_seats,has_bicycle,transport_type,o_station_name,o_station_lat,o_station_lon,d_station_name,d_station_lat,d_station_lon
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,True,True,False,bus,Avenue Georges Pompidou,47.896911,1.854277,Station Les Sabines,43.583932,3.860233
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},...,False,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500
49,6795074,13,767.0,396.0,2017-10-06 05:25:00,2017-10-06 16:40:00,1990,2017-10-01 01:04:12.45+00,"{932,932}",{13},...,False,False,False,bus,Massy-Palaiseau,48.727660,2.264629,Gare Lille-Europe,50.638756,3.076675


### Calcul la distance du voyage

In [21]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians

def haversine_vectorize(lon1, lat1, lon2, lat2):
 
    #lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    lon1 = radians(lon1)
    lat1 = radians(lat1)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    
    newlon = lon2 - lon1
    newlat = lat2 - lat1
 
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
 
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [22]:
ticket_provider.dtypes

id                               int64
company                          int64
o_station                      float64
d_station                      float64
departure_ts            datetime64[ns]
arrival_ts              datetime64[ns]
price_in_cents                   int64
search_ts                       object
middle_stations                 object
other_companies                 object
o_city                           int64
d_city                           int64
duration                       float64
company_id                       int64
provider_id                     object
name                            object
provider_name                   object
has_wifi                        object
has_plug                        object
has_adjustable_seats            object
has_bicycle                     object
transport_type                  object
o_station_name                  object
o_station_lat                  float64
o_station_lon                  float64
d_station_name           

In [23]:
new_column = []
for index,row in ticket_provider.iterrows():
    lat1 = row['o_station_lat']
    lon1 = row['o_station_lon']
    lat2 = row['d_station_lat']
    lon2 = row['d_station_lon']
    
    value = haversine_vectorize(lat1, lon1, lat2, lon2) 
    new_column.append(value) 

# Supprimer la colonne 'distance_in_km' si elles existent dèja
ticket_provider = ticket_provider.drop(['distance_in_km'], axis=1, errors='ignore')
ticket_provider.insert(len(ticket_provider.columns), "distance_in_km",new_column)

# Test
ticket_provider.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,has_adjustable_seats,has_bicycle,transport_type,o_station_name,o_station_lat,o_station_lon,d_station_name,d_station_lat,d_station_lon,distance_in_km
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,True,False,bus,Avenue Georges Pompidou,47.896911,1.854277,Station Les Sabines,43.583932,3.860233,528.019650
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",...,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",...,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},...,True,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191
49,6795074,13,767.0,396.0,2017-10-06 05:25:00,2017-10-06 16:40:00,1990,2017-10-01 01:04:12.45+00,"{932,932}",{13},...,False,False,bus,Massy-Palaiseau,48.727660,2.264629,Gare Lille-Europe,50.638756,3.076675,230.533463


#### Ajouter la colonne distance_level

In [24]:
# Ajouter la colonne distance_level (0-200km, 201-800km, 800-2000km, 2000+km)
level_column = []
for index,row in ticket_provider.iterrows():
    distance_valeur = row['distance_in_km'] 
    if (distance_valeur <= 200):
        disance_level = '0-200'
    elif (distance_valeur > 200) & (distance_valeur <= 800): 
        disance_level = '201-800'
    elif (distance_valeur > 800) & (distance_valeur <= 2000):
        disance_level = '801-2000'
    else:
        disance_level = '2000+'
    level_column.append(disance_level) 

# Supprimer la colonne 'distance_in_km' si elles existent dèja
ticket_provider = ticket_provider.drop(['distance_level_in_km'], axis=1, errors='ignore')
ticket_provider.insert(len(ticket_provider.columns), "distance_level_in_km",level_column)

# Test
#ticket_provider.dtypes
ticket_provider.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,has_bicycle,transport_type,o_station_name,o_station_lat,o_station_lon,d_station_name,d_station_lat,d_station_lon,distance_in_km,distance_level_in_km
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00,2017-10-14 06:55:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,False,bus,Avenue Georges Pompidou,47.896911,1.854277,Station Les Sabines,43.583932,3.860233,528.019650,201-800
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 21:24:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",...,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191,201-800
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00,2017-10-14 11:02:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",...,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191,201-800
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00,2017-10-14 19:32:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},...,False,train,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,527.400191,201-800
49,6795074,13,767.0,396.0,2017-10-06 05:25:00,2017-10-06 16:40:00,1990,2017-10-01 01:04:12.45+00,"{932,932}",{13},...,False,bus,Massy-Palaiseau,48.727660,2.264629,Gare Lille-Europe,50.638756,3.076675,230.533463,201-800


#### Différence de PRIX moyen selon le train, le bus et le covoit selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km

In [25]:
# Caculer le prix min, moyen et max par distance trajet selon le train, le bus et le covoit 
prix_trajet_distance = ticket_provider.groupby(['distance_level_in_km','transport_type'])["price_in_cents"].agg(['min', 'mean', 'max'])

## Changer les noms des ces nouvelles colonnes
prix_trajet_distance.rename(columns={'min':'min_price_in_cents', 'mean':'mean_price_in_cents', 'max':'max_price_in_cents'}, inplace=True)

# Round 
decimals = pd.Series([0], index=['mean_price_in_cents'])
prix_trajet_distance = prix_trajet_distance.round(decimals)

#Test
prix_trajet_distance
# type(trajet_distance)

min_price_in_cents  mean_price_in_cents  \
distance_level_in_km transport_type                                            
0-200                bus                            850               2106.0   
                     train                          490               3264.0   
2000+                bus                           6900               9639.0   
201-800              bus                           1000               3391.0   
                     train                         1400               8741.0   
801-2000             bus                           1980               5780.0   
                     train                         4550              15652.0   

                                     max_price_in_cents  
distance_level_in_km transport_type                      
0-200                bus                          22900  
                     train                        25100  
2000+                bus                          17400  
201-800              bus                          22480  
                     train                        38550  
801-2000             bus                          17980  
                     train                        37550

#### Différence de DUREE moyen selon le train, le bus et le covoit selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km

In [26]:
# Caculer le durée min, moyen et max par distance trajet selon le train, le bus et le covoit selon ...
duration_trajet_distance = ticket_provider.groupby(['distance_level_in_km','transport_type'])['duration'].agg(['min', 'mean', 'max'])

## Changer les noms des ces nouvelles colonnes
duration_trajet_distance.rename(columns={'min':'min_duration_in_seconds', 'mean':'mean_duration_in_seconds', 'max':'max_duration_in_seconds'}, inplace=True)

duration_trajet_distance.head()
# Round 
decimals = pd.Series([0], index=['mean_duration_in_seconds'])
duration_trajet_distance = duration_trajet_distance.round(decimals)

#Test
duration_trajet_distance
# type(trajet_distance)

min_duration_in_seconds  \
distance_level_in_km transport_type                            
0-200                bus                              3900.0   
                     train                            2340.0   
2000+                bus                             89700.0   
201-800              bus                              8940.0   
                     train                            5340.0   
801-2000             bus                             31800.0   
                     train                           14520.0   

                                     mean_duration_in_seconds  \
distance_level_in_km transport_type                             
0-200                bus                              36154.0   
                     train                            15435.0   
2000+                bus                             147954.0   
201-800              bus                              50429.0   
                     train                            26658.0   
801-2000             bus                              96947.0   
                     train                            45859.0   

                                     max_duration_in_seconds  
distance_level_in_km transport_type                           
0-200                bus                           1143900.0  
                     train                          122640.0  
2000+                bus                            726300.0  
201-800              bus                           1774260.0  
                     train                          174420.0  
801-2000             bus                           1296000.0  
                     train                          157860.0

#### Jointure df prix_trajet_distance et duration_trajet_distance pour d'avoir le resultat final

In [27]:
trajet_distance = duration_trajet_distance.join(prix_trajet_distance, on=['distance_level_in_km','transport_type'])
#Test
trajet_distance


min_duration_in_seconds  \
distance_level_in_km transport_type                            
0-200                bus                              3900.0   
                     train                            2340.0   
2000+                bus                             89700.0   
201-800              bus                              8940.0   
                     train                            5340.0   
801-2000             bus                             31800.0   
                     train                           14520.0   

                                     mean_duration_in_seconds  \
distance_level_in_km transport_type                             
0-200                bus                              36154.0   
                     train                            15435.0   
2000+                bus                             147954.0   
201-800              bus                              50429.0   
                     train                            26658.0   
801-2000             bus                              96947.0   
                     train                            45859.0   

                                     max_duration_in_seconds  \
distance_level_in_km transport_type                            
0-200                bus                           1143900.0   
                     train                          122640.0   
2000+                bus                            726300.0   
201-800              bus                           1774260.0   
                     train                          174420.0   
801-2000             bus                           1296000.0   
                     train                          157860.0   

                                     min_price_in_cents  mean_price_in_cents  \
distance_level_in_km transport_type                                            
0-200                bus                            850               2106.0   
                     train                          490               3264.0   
2000+                bus                           6900               9639.0   
201-800              bus                           1000               3391.0   
                     train                         1400               8741.0   
801-2000             bus                           1980               5780.0   
                     train                         4550              15652.0   

                                     max_price_in_cents  
distance_level_in_km transport_type                      
0-200                bus                          22900  
                     train                        25100  
2000+                bus                          17400  
201-800              bus                          22480  
                     train                        38550  
801-2000             bus                          17980  
                     train                        37550